In [1]:
#█    ▀  █▀▀▄ █▀▀█ █▀▀█ █▀▀█ █  █ █▀▀ 
#█   ▀█▀ █▀▀▄ █▄▄▀ █▄▄█ █▄▄▀ █▄▄█ ▀▀█ 
#▀▀▀ ▀▀▀ ▀▀▀  ▀ ▀▀ ▀  ▀ ▀ ▀▀ ▄▄▄█ ▀▀▀ 
#

import time
import pandas as pd
import datetime

from numpy import log, sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import rand 
seed = 1

import pyspark.sql.functions as F

from pyspark.sql.types import *
from pyspark.sql.functions import col

from pyspark.sql import SparkSession
import warnings

#variable for skipping all the plotting code for debugging processing
plotting = False
start_time = time.perf_counter()
start_time_og = time.perf_counter()

#█▀ █▀█ ▄▀█ █▀█ █▄▀ 
#▄█ █▀▀ █▀█ █▀▄ █ █ 
# spark set up

#removes warnings
warnings.filterwarnings("ignore")

# create a spark session (which will run spark jobs)
spark = SparkSession.builder.getOrCreate()

# makes the outputted df nicely formatted
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)



#█▀▄ ▄▀█ ▀█▀ ▄▀█    █▀▄ █ █▀█ 
#█▄▀ █▀█  █  █▀█    █▄▀ █ █▀▄ 
#

#main data directory
data_dir = "/mnt/e/2021/Applied Data Science/Project 1/Data/"


#▀█▀ ▄▀█ ▀▄▀ █    █▀▄ ▄▀█ ▀█▀ ▄▀█ 
# █  █▀█ █ █ █    █▄▀ █▀█  █  █▀█ 
#

#file location
file_base = "_tripdata_"

#indexs for each of the main data set types
ywl_i = 0 
grn_i = 1
fhv_i = 0
hvf_i = 1
taxi_types_n = 2

#names related to each cab type in order
taxi_file_names = ["fhv","fhvhv"]
full_names = ["for higher vheciles", "for higher high volume"]

#column names for each cab type in order
column_names = [
#    ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'],
#    ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
    ['dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag'],
    ['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag']
]

#2018 fhv files which was giving me trouble so added this schema for it
bad_fhv_schema = [2, 2018, ['Pickup_DateTime','DropOff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag','dispatching_base_number']]


#aliases for pickup time and dropoff time
pickup_time = ['pickup_datetime','pickup_datetime','Pickup_DateTime']
dropoff_time = ['dropoff_datetime','dropoff_datetime','DropOff_datetime']

#irrelavent collumns that can be dropped
irrelavent_columns = ['dispatching_base_number','DOtime','hvfhs_license_num','dispatching_base_num','VendorID','store_and_fwd_flag','extra','MTA_tax','improvement_surcharge','tip_amount','tolls_amount', 'ehail_fee', 'improvement_surcharge', 'congestion_surcharge']


#▀█▀ ▄▀█ ▀▄▀ █    █   █▀█ █▀▀ ▄▀█ ▀█▀ █ █▀█ █▄ █ █▀ 
# █  █▀█ █ █ █    █▄▄ █▄█ █▄▄ █▀█  █  █ █▄█ █ ▀█ ▄█ 
#

#location processing variables
#locations that will be removed as considered invalid
bad_locations = [1,132,138,264,265]

#location of the location id file
location_file = "taxi+_zone_lookup.csv"

#list of boroughs names as in the location file
boroughs = ["Bronx","Brooklyn","Manhattan","Queens","Staten Island","EWR","Unknown"]


#█▀▄ ▄▀█ ▀█▀ █▀▀    █▀▄ █▀▀ █▀▀ █▀ 
#█▄▀ █▀█  █  ██▄    █▄▀ ██▄ █▀  ▄█ 
#
months_in_year = 12

#years of data used
years = [2019, 2020]

#feb 2019 was when fhvhv file started
fhvhv_started = [2,2019] 

#late feb 2020 was when the first case of covid was observed in NYC
covid_started = [2,2020] 

#preprocessing arrays

#hours each day to seperate each count
day_time_breakdown = [0,6,12,18]

#day of the year covid started on
start_day = int(datetime.datetime.strptime(
    " ".join([str(int) for int in covid_started]), 
    "%m %Y").__format__("%j"))

#pretty self explanitory
days_in_2020 = 366


#█▀█ █ █ ▀█▀ █   █ █▀▀ █▀█ █▀ 
#█▄█ █▄█  █  █▄▄ █ ██▄ █▀▄ ▄█ 
#

outlier_percent = (1-.99)/2
fare_amount_graph_buffer = 15
percentile_accuracy = 0.00001


#█▀▀ █▀█ █ █ █ █▀▄    █▀▄ █▀▀ █▀▀ █▀ 
#█▄▄ █▄█ ▀▄▀ █ █▄▀    █▄▀ ██▄ █▄▄ ▄█ 
#

#files
covid_restrictions_file = "covid_restrictions.csv"
covid_cases_file = "COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"

#list of boroughs as named in the covid file (same order)
covid_boroughs =["bx","bk","mn","qn","si"]

#covid data of interest
covid_of_interest = ['case_count', 'probable_case_count','hospitalized_count', 'death_count', 'probable_death_count',
       'case_count_7day_avg', 'all_case_count_7day_avg', 'hospitalized_count_7day_avg',
       'death_count_7day_avg', 'all_death_count_7day_avg']



processed_data_file = "_processed_data_"
output_file = "combined_taxi_data.csv"



Picked up _JAVA_OPTIONS: -Xmx6000m
Picked up _JAVA_OPTIONS: -Xmx6000m
21/08/09 11:41:32 WARN Utils: Your hostname, DESKTOP-GN6J0KT resolves to a loopback address: 127.0.1.1; using 172.24.141.130 instead (on interface eth0)
21/08/09 11:41:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/09 11:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/09 11:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#█▀▀ █▀▀ █  █ █▀▀ █▀▄▀█ █▀▀█    █▀▀ █▀▀ ▀▀█▀▀ █  █ █▀▀█ 
#▀▀█ █   █▀▀█ █▀▀ █ ▀ █ █▄▄█    ▀▀█ █▀▀   █   █  █ █▄▄█ 
#▀▀▀ ▀▀▀ ▀  ▀ ▀▀▀ ▀   ▀ ▀  ▀    ▀▀▀ ▀▀▀   ▀    ▀▀▀ █    
#

ints = ('SR_Flag','VendorID', 'trip_type', 'passenger_count', 'RateCodeID', 'RatecodeID','payment_type','PULocationID','DOLocationID')
doubles = ('trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','ehail_fee',
           'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount','congestion_surcharge')
strings = ('dispatching_base_number','store_and_fwd_flag','dispatching_base_num','hvfhs_license_num')
dtimes = ('DropOff_datetime','Pickup_DateTime','tpep_pickup_datetime', 'tpep_dropoff_datetime', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime')

dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})





#schema generation used from the tutes
schema = []
for index in range(0,taxi_types_n):
    schema.append(StructType())
    for column in column_names[index]:
        schema[index].add(column, # column name
                   dtypes[column], # data type
                   True # is nullable?
                  )
        
        
        
#schema for the bad fhv file
bad_fhv_schema.append(StructType())
for column in bad_fhv_schema[2]:
    bad_fhv_schema[3].add(column,
                         dtypes[column],
                         True)

In [3]:
#██╗ ███╗   ███╗ ██████╗   ██████╗  ██████╗  ████████╗ ██╗ ███╗   ██╗  ██████╗  
#██║ ████╗ ████║ ██╔══██╗ ██╔═══██╗ ██╔══██╗ ╚══██╔══╝ ██║ ████╗  ██║ ██╔════╝  
#██║ ██╔████╔██║ ██████╔╝ ██║   ██║ ██████╔╝    ██║    ██║ ██╔██╗ ██║ ██║  ███╗ 
#██║ ██║╚██╔╝██║ ██╔═══╝  ██║   ██║ ██╔══██╗    ██║    ██║ ██║╚██╗██║ ██║   ██║ 
#██║ ██║ ╚═╝ ██║ ██║      ╚██████╔╝ ██║  ██║    ██║    ██║ ██║ ╚████║ ╚██████╔╝ 
#╚═╝ ╚═╝     ╚═╝ ╚═╝       ╚═════╝  ╚═╝  ╚═╝    ╚═╝    ╚═╝ ╚═╝  ╚═══╝  ╚═════╝  
# imports all the relavent csv files to 


#▀█▀ ▄▀█ ▀▄▀ █ 
# █  █▀█ █ █ █ 
#
def read_csv(year, month, taxi_i, schema_used, drop_index):
    file = data_dir + taxi_file_names[taxi_i] + file_base + str(year) + "-" + str(month).zfill(2) + ".csv"
    print(file)
    return (
        spark.read.csv(
            file, 
            header=True, 
            schema=schema_used)\
        .withColumnRenamed(pickup_time[drop_index],"PUtime")# rename the time col
        .withColumnRenamed(dropoff_time[drop_index],"DOtime")# rename other time col
        .drop(*irrelavent_columns)
        #.sample(False, 0.1, seed=seed).limit(10000) #testing line        
    )


#spark data frame matrix
dfm = {}

for year in years:
    dfm[year] = {}
    
    #loops over the months in years
    for month in range(covid_started[0], months_in_year + 1): 
        dfm[year][month] = {}
    
        #for each taxi variant
        for taxi_i in range(0,len(taxi_file_names)):
            dfm[year][month][taxi_i] = 0
           

            
            if(bad_fhv_schema[1] == year and taxi_i == fhv_i):
                
                schema_used = bad_fhv_schema[3]
                drop_index = 3
            else:
                schema_used = schema[taxi_i]
                drop_index = taxi_i
                
                    
                    
            
            #small condition as fhvhv didnt always exist
            if( (month >= fhvhv_started[0] and year >= fhvhv_started[1]) or taxi_i != hvf_i):
                
                #prints that the functions is processing
                print("importing " + taxi_file_names[taxi_i] + "    \t" + str(year) + "-" + str(month).zfill(2))                
                
                #reads the first dataframe
                dfm[year][month][taxi_i] = read_csv(year, month, taxi_i, schema_used, drop_index)
            
            else:
                dfm[year][month][taxi_i] = None

for year in years:
    for taxi_i in range(0,len(taxi_file_names)):
        if( dfm[year][month][taxi_i] ):
            dfm[year][month][taxi_i] = dfm[year][month][taxi_i]#.orderBy(rand(seed=seed))).limit(1000)

importing fhv    	2019-02
/mnt/e/2021/Applied Data Science/Project 1/Data/fhv_tripdata_2019-02.csv
importing fhvhv    	2019-02
/mnt/e/2021/Applied Data Science/Project 1/Data/fhvhv_tripdata_2019-02.csv
importing fhv    	2019-03
/mnt/e/2021/Applied Data Science/Project 1/Data/fhv_tripdata_2019-03.csv
importing fhvhv    	2019-03
/mnt/e/2021/Applied Data Science/Project 1/Data/fhvhv_tripdata_2019-03.csv
importing fhv    	2019-04
/mnt/e/2021/Applied Data Science/Project 1/Data/fhv_tripdata_2019-04.csv
importing fhvhv    	2019-04
/mnt/e/2021/Applied Data Science/Project 1/Data/fhvhv_tripdata_2019-04.csv
importing fhv    	2019-05
/mnt/e/2021/Applied Data Science/Project 1/Data/fhv_tripdata_2019-05.csv
importing fhvhv    	2019-05
/mnt/e/2021/Applied Data Science/Project 1/Data/fhvhv_tripdata_2019-05.csv
importing fhv    	2019-06
/mnt/e/2021/Applied Data Science/Project 1/Data/fhv_tripdata_2019-06.csv
importing fhvhv    	2019-06
/mnt/e/2021/Applied Data Science/Project 1/Data/fhvhv_tripdata_20

In [4]:

print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

14.151922099998046 seconds have elapsed
14.152112799994939 seconds have elapsed total


In [5]:
#█▀▀ ▄▀█ █▀ █▀▀ █▀ 
#█▄▄ █▀█ ▄█ ██▄ ▄█ 
# #importing covid case data
covid_cases = pd.read_csv(data_dir + covid_cases_file)

#makes the names of columns lowercase
covid_cases.columns = covid_cases.columns.str.lower()

#makes the date column datetime object
covid_cases['date_of_interest'] = covid_cases['date_of_interest']\
    .transform(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

#renaming some badly named columns
covid_cases['probable_death_count'] = covid_cases['death_count_probable']
covid_cases['hospitalized_count_7day_avg'] = covid_cases['hosp_count_7day_avg']
covid_cases = covid_cases.drop(['death_count_probable','hosp_count_7day_avg'],axis=1)

In [6]:
#█▀█ █▀▀ █▀ ▀█▀ █▀█ █ █▀▀ ▀█▀ █ █▀█ █▄ █ █▀ 
#█▀▄ ██▄ ▄█  █  █▀▄ █ █▄▄  █  █ █▄█ █ ▀█ ▄█ 
#

#importing covid restrictions data
covid_restrictions = pd.read_csv(data_dir + covid_restrictions_file)

In [7]:
if plotting:
    
    #if we are plotting, gets some data to plot
    dft = dfm[2019][5][ywl_i].sample(False, 0.1, seed=0).limit(10000)
    
    #extracting all the fare amounts
    data = dft.select('fare_amount').toPandas()['fare_amount']
    
    # apply a log transformation for all x non-zero x points, else 0
    dataL = data.apply(lambda x: log(x) if x else 0)

In [8]:
if plotting:
    
    #plots the fare amount as a histogram (no log)
    sns.distplot(data, bins= round((data.max()-data.min())/(2)))
    plt.xlim(data.quantile(outlier_percent)-fare_amount_graph_buffer, data.quantile(1-outlier_percent)+fare_amount_graph_buffer)
    plt.show()

In [9]:
if plotting:
    
    #plots the log of the fare amount as a histogram
    plt.xlim(dataL.quantile(outlier_percent)-log(fare_amount_graph_buffer), dataL.quantile(1-outlier_percent)+log(fare_amount_graph_buffer))
    sns.distplot(dataL, bins= round((dataL.max()-dataL.min()))*2)
    plt.show()
    
    #tried other plots and they looked like shit

In [10]:
if plotting:
    
    #extracting all the 
    data = dft.select('trip_distance').toPandas()['trip_distance']
    
    # apply a log transformation for all x non-zero x points, else 0
    dataL = data.apply(lambda x: log(x) if x else 0)

In [11]:
if plotting:
    
    #fare amount as a historgram
    plt.xlim(data.quantile(outlier_percent)-fare_amount_graph_buffer, data.quantile(1-outlier_percent)+fare_amount_graph_buffer)
    sns.distplot(data, bins= round((data.max()-data.min())*1))
    plt.show()

In [12]:
if plotting:
    
    #log( fare amount) as a histogram
    plt.xlim(dataL.quantile(outlier_percent)-log(fare_amount_graph_buffer), dataL.quantile(1-outlier_percent)+log(fare_amount_graph_buffer))
    sns.distplot(dataL, bins= round((dataL.max()-dataL.min()))*5)
    plt.show()

In [13]:
if plotting:
    
    #initial plot of trip vs cost data, very noisy, alot to remove
    data = dft.select('fare_amount','trip_distance').toPandas()
    data.plot.scatter(y='fare_amount',x='trip_distance')

In [14]:
#███████╗ ██╗ ██╗      ████████╗ ███████╗ ██████╗  ██╗ ███╗   ██╗  ██████╗  
#██╔════╝ ██║ ██║      ╚══██╔══╝ ██╔════╝ ██╔══██╗ ██║ ████╗  ██║ ██╔════╝  
#█████╗   ██║ ██║         ██║    █████╗   ██████╔╝ ██║ ██╔██╗ ██║ ██║  ███╗ 
#██╔══╝   ██║ ██║         ██║    ██╔══╝   ██╔══██╗ ██║ ██║╚██╗██║ ██║   ██║ 
#██║      ██║ ███████╗    ██║    ███████╗ ██║  ██║ ██║ ██║ ╚████║ ╚██████╔╝ 
#╚═╝      ╚═╝ ╚══════╝    ╚═╝    ╚══════╝ ╚═╝  ╚═╝ ╚═╝ ╚═╝  ╚═══╝  ╚═════╝  
# removing values that dont make sense, etc


#█   █▀█ █▀▀ ▄▀█ ▀█▀ █ █▀█ █▄ █ 
#█▄▄ █▄█ █▄▄ █▀█  █  █ █▄█ █ ▀█ 
# removing invalid locations

#checks if location id is in the bad location list
@F.udf("boolean")
def bad_location(LocationID):
    if(LocationID):
        return (int(LocationID) not in bad_locations)
    else:
        return False

#for each year and each taxi 
for year in years:
    for month in range(covid_started[0], months_in_year + 1): 
        for taxi_i in range(0,len(taxi_file_names)):
            if( dfm[year][month][taxi_i] ):
                
                print("processing " + str(year) + "-" + str(month) + " " + taxi_file_names[taxi_i])
                dfm[year][month][taxi_i] = dfm[year][month][taxi_i]\
                    .filter(
                        (bad_location(col("PULocationID"))) &
                        (bad_location(col("DOLocationID")))
                           )

processing 2019-2 fhv
processing 2019-2 fhvhv
processing 2019-3 fhv
processing 2019-3 fhvhv
processing 2019-4 fhv
processing 2019-4 fhvhv
processing 2019-5 fhv
processing 2019-5 fhvhv
processing 2019-6 fhv
processing 2019-6 fhvhv
processing 2019-7 fhv
processing 2019-7 fhvhv
processing 2019-8 fhv
processing 2019-8 fhvhv
processing 2019-9 fhv
processing 2019-9 fhvhv
processing 2019-10 fhv
processing 2019-10 fhvhv
processing 2019-11 fhv
processing 2019-11 fhvhv
processing 2019-12 fhv
processing 2019-12 fhvhv
processing 2020-2 fhv
processing 2020-2 fhvhv
processing 2020-3 fhv
processing 2020-3 fhvhv
processing 2020-4 fhv
processing 2020-4 fhvhv
processing 2020-5 fhv
processing 2020-5 fhvhv
processing 2020-6 fhv
processing 2020-6 fhvhv
processing 2020-7 fhv
processing 2020-7 fhvhv
processing 2020-8 fhv
processing 2020-8 fhvhv
processing 2020-9 fhv
processing 2020-9 fhvhv
processing 2020-10 fhv
processing 2020-10 fhvhv
processing 2020-11 fhv
processing 2020-11 fhvhv
processing 2020-12 fhv
p

In [15]:
for year in years:
    for taxi_i in [fhv_i, hvf_i]:
        for month in range(covid_started[0], months_in_year + 1): 
            if( dfm[year][month][taxi_i] ):
                dfm[year][month][taxi_i] = dfm[year][month][taxi_i]\
                .filter(
                    (col("SR_flag").isNull())
                ).drop("SR_flag")
            

In [16]:
#the following processing only makes sense for yellow and green taxis


#█▀█ ▄▀█ █▀ █▀ █▀▀ █▄ █ █▀▀ █▀▀ █▀█ 
#█▀▀ █▀█ ▄█ ▄█ ██▄ █ ▀█ █▄█ ██▄ █▀▄ 
# removing invalid passenger counts

#█▀█ ▄▀█ ▀█▀ █▀▀ █▀ 
#█▀▄ █▀█  █  ██▄ ▄█ 
# only keep standard rates

#█▀█ ▄▀█ █▄█ █▀▄▀█ █▀▀ █▄ █ ▀█▀ 
#█▀▀ █▀█  █  █ ▀ █ ██▄ █ ▀█  █  
# payment types which arent helpful
#for year in years:
#    for month in range(covid_started[0], months_in_year + 1): 
#        for taxi_i in [ywl_i, grn_i]:
#            if( dfm[year][month][taxi_i] ):
#                dfm[year][month][taxi_i] = dfm[year][month][taxi_i]\
#                .filter(
#                    (
#                        (col("payment_type") == 1) | 
#                        (col("payment_type") == 0)
#                    ) &
#                    (col("passenger_count") > 0) &
#                    (col("RateCodeID") == 1)
#                )#.drop("RateCodeID","passenger_count","payment_type")
#                 
                
                


In [17]:
#█▀▀ █▀█ █▀ ▀█▀    ▄▀█ █▀▄▀█ █▀█ █ █ █▄ █ ▀█▀ 
#█▄▄ █▄█ ▄█  █     █▀█ █ ▀ █ █▄█ █▄█ █ ▀█  █  
# cost are sometimes bad, graphing what it looks like atm

if plotting:
    
    dft = dfm[2019][5][ywl_i].sample(False, 0.1, seed=0).limit(10000)
    
    #extracting all the fare amounts
    data = dft.select('fare_amount').select('fare_amount').toPandas()['fare_amount']
    
    #checking max and min
    print([data.min(),data.max()])
    
    #taking the log transform 
    data = data.apply(lambda x: log(x) if x else 0)
    
    #plotting the log of fare amounts to view distribution
    plt.xlim(data.quantile(outlier_percent)-log(fare_amount_graph_buffer), data.quantile(1-outlier_percent)+log(fare_amount_graph_buffer))
    sns.distplot(data, bins= round((data.max()-data.min()))*4)
    plt.show()
    
    
    #dfm[2019][ywl_i].select("fare_amount").summary().show()

In [18]:
#▀█▀ █▀█ █ █▀█    █▀▄ █ █▀ ▀█▀ ▄▀█ █▄ █ █▀▀ █▀▀ 
# █  █▀▄ █ █▀▀    █▄▀ █ ▄█  █  █▀█ █ ▀█ █▄▄ ██▄ 
#

if plotting:
        
    #extracting all the fare amounts
    data = dft.select('trip_distance').toPandas()['trip_distance']
    
    #checking max and min
    print([data.min(),data.max()])
    
    #taking the log transform 
    data = data.apply(lambda x: log(x) if x else 0)
    
    #plotting the log of fare amounts to view distribution
    plt.xlim(data.quantile(outlier_percent)-log(fare_amount_graph_buffer), data.quantile(1-outlier_percent)+log(fare_amount_graph_buffer))
    sns.distplot(data, bins= round((data.max()-data.min()))*4)
    plt.show()
    
    
    #dfm[2019][ywl_i].select("fare_amount").summary().show()

In [19]:
if plotting:
    
    #plotting to view fare cost vs trip distance to see if there are many outliers, there are a few
    data = dft.select('fare_amount','trip_distance').toPandas()
    data.plot.scatter(y='fare_amount',x='trip_distance')

In [20]:
if plotting:
        
    #plots a new fare amount vs trip distance graph to see if the majority of outliers have been removed
    dft = dfm[2020][5][grn_i].sample(False, 0.1, seed=0).limit(10000)
    data = dft.select('fare_amount','trip_distance').toPandas()
    data.plot.scatter(y='fare_amount',x='trip_distance')

In [21]:
#prints how much time has elapsed
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

1.6110017000028165 seconds have elapsed
15.763895600000978 seconds have elapsed total


In [22]:
#ass fare amount is obviously dependent on trip distance, the one quantiles cant be calculated after processing the other set
#so they must both be processed in the same filter

#making quantile array
qt = {}

#for each year, create sub array
#for year in years:
#    qt[year] = {}
#    
#    #for each taxi create another sub array
#    for taxi_i in [ywl_i, grn_i]:
#        qt[year][taxi_i] = {}
#        
#        qt_combined = [[0,0],[0,0]]
#        count = 0
#        
#        #for each month
#        for month in range(covid_started[0], months_in_year + 1): 
#            
#            #set initally as null
#            qt[year][taxi_i][month] = None
#            
#            #checks data exists
#            if( dfm[year][month][taxi_i] ):
#                
#                print("processing " + str(year) + "-" + str(month) + " " + taxi_file_names[taxi_i])
#                
#                qt[year][taxi_i][month] = dfm[year][month][taxi_i].approxQuantile(
#                    ["fare_amount",'trip_distance'], 
#                    [outlier_percent,1-outlier_percent], 
#                    percentile_accuracy
#                )
#                
#                for i in range(0,2):
#                    for j in range(0,2):
#                        qt_combined[i][j] += qt[year][taxi_i][month][i][j]
#                    
#                count += 1
#                print(qt[year][taxi_i][month])
#                
#                
#        for i in range(0,2):
#            for j in range(0,2):
#                qt_combined[i][j] = qt_combined[i][j]/count
#
#        
#        #2.5 is the miniumum fare_amount so if the fare quantile was lower than the minimum, it is raised
#        if qt_combined[0][0] < 2.5 :
#            qt_combined[0][0] = 2.5
#                
#        #less than a 0km trip shouldnt be counted so min of distance it set to 0 if it was lower.
#        if qt_combined[1][0] < 0 :
#            qt_combined[1][0] = 0
#                         
#        print("combined quantile:")
#        print(qt_combined)               
#            
#            
#        #for each month
#        for month in range(covid_started[0], months_in_year + 1): 
#                                
#                    
#                    
#                #only keeps the middle percent% (depends on what percent was decided) to remove outliers 
#                dfm[year][month][taxi_i] = dfm[year][month][taxi_i].filter(
#                    (col("fare_amount") > qt_combined[0][0]) & 
#                    (col("fare_amount") < qt_combined[0][1]) &                
#                    (col('trip_distance') > qt_combined[1][0]) & 
#                    (col('trip_distance') < qt_combined[1][1])
#                )#.drop("fare_amount","total_amount"'trip_distance')
#

In [23]:
#prints how much time has elapsed
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

0.027815399997052737 seconds have elapsed
15.7920862999963 seconds have elapsed total


In [24]:
print(qt)

{}


In [25]:
#qt_combined

In [26]:
if plotting:
    
    dft = dfm[2019][5][ywl_i].sample(False, 0.1, seed=0).limit(10000)
    
    #plotting to view fare cost vs trip distance to see if there are many outliers, there are a few
    data = dft.select('fare_amount','trip_distance').toPandas()
    data.plot.scatter(y='fare_amount',x='trip_distance')

In [27]:
if plotting:
        
    #plots a new fare amount vs trip distance graph to see if the majority of outliers have been removed
    dft = dfm[2020][5][grn_i].sample(False, 0.1, seed=0).limit(10000)
    data = dft.select('fare_amount','trip_distance').toPandas()
    data.plot.scatter(y='fare_amount',x='trip_distance')

In [28]:
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

0.05963550000160467 seconds have elapsed
15.852144500000577 seconds have elapsed total


In [29]:
#██████╗  ██████╗  ███████╗ ██████╗  ██████╗   ██████╗   ██████╗ ███████╗ ███████╗ ███████╗ ██╗ ███╗   ██╗  ██████╗  
#██╔══██╗ ██╔══██╗ ██╔════╝ ██╔══██╗ ██╔══██╗ ██╔═══██╗ ██╔════╝ ██╔════╝ ██╔════╝ ██╔════╝ ██║ ████╗  ██║ ██╔════╝  
#██████╔╝ ██████╔╝ █████╗   ██████╔╝ ██████╔╝ ██║   ██║ ██║      █████╗   ███████╗ ███████╗ ██║ ██╔██╗ ██║ ██║  ███╗ 
#██╔═══╝  ██╔══██╗ ██╔══╝   ██╔═══╝  ██╔══██╗ ██║   ██║ ██║      ██╔══╝   ╚════██║ ╚════██║ ██║ ██║╚██╗██║ ██║   ██║ 
#██║      ██║  ██║ ███████╗ ██║      ██║  ██║ ╚██████╔╝ ╚██████╗ ███████╗ ███████║ ███████║ ██║ ██║ ╚████║ ╚██████╔╝ 
#╚═╝      ╚═╝  ╚═╝ ╚══════╝ ╚═╝      ╚═╝  ╚═╝  ╚═════╝   ╚═════╝ ╚══════╝ ╚══════╝ ╚══════╝ ╚═╝ ╚═╝  ╚═══╝  ╚═════╝  
# Adding generated features and changing forms of other features

#█▀▄ ▄▀█ ▀█▀ █▀▀    ▄▀█ █▄ █ █▀▄    ▀█▀ █ █▀▄▀█ █▀▀ 
#█▄▀ █▀█  █  ██▄    █▀█ █ ▀█ █▄▀     █  █ █ ▀ █ ██▄ 
# adding in binned date and time

#UDFs can only have one output
#this UDF bins the time of the day into the specified groups with int output between 1 - n being the group number
@F.udf("int")
def find_part_of_day(time):
    
    #finds hour of the day once to speed code a bit
    hour = int(time.__format__("%H"))
    
    #checks which part of the day that hour should be in
    if hour < day_time_breakdown[1]:
        return 0
    elif hour < day_time_breakdown[2]:
        return 1
    elif hour < day_time_breakdown[3]:
        return 2
    else:
        return 3

#this UDF extracts the week of the year from the date time column
#years are captured in the matrix level and months will become irrelavent shortly
@F.udf("int")
def find_week_of_year(time):
    return int(time.__format__("%U"))

#the individual days can then be subsorted by looking at the day of the week
@F.udf("int")
def find_day_of_week(time):
    return int(time.__format__("%w"))

#testdf = dfm[2020][ywl_i].withColumn("week_of_year", find_week_of_year(col("PUtime"))).withColumn("day_of_week", find_day_of_week(col("PUtime"))).withColumn("part_of_day", find_part_of_day(col("PUtime")))

In [30]:
#█   █▀█ █▀▀ ▄▀█ ▀█▀ █ █▀█ █▄ █ 
#█▄▄ █▄█ █▄▄ █▀█  █  █ █▄█ █ ▀█ 
# adding in borough number

#first we need to get the location id to borough id matrix 

#reads the location id file
borough_codes = pd.read_csv(data_dir + location_file)

#finds the 'id' for each location interms of the 5 boroughs
codes = []
for index, row in borough_codes.iterrows():
    codes.append(boroughs.index(row["Borough"]))

#adds said list to the dataframe    
borough_codes['Borough_code'] = pd.Series(codes, index = borough_codes.index)

#location 
@F.udf("int")
def find_borough_code(location):
    return int(borough_codes.loc[borough_codes["LocationID"] == int(location)]["Borough_code"].values[0])

In [31]:
#applying location and date row transformations

for year in years:
    for month in range(covid_started[0], months_in_year + 1): 
        for taxi_i in range(0,len(taxi_file_names)):
            if( dfm[year][month][taxi_i] ):
                
                print("processing " + str(year) + " " + taxi_file_names[taxi_i])
                dfm[year][month][taxi_i] = dfm[year][month][taxi_i]\
                    .withColumn("PU_borough_code", find_borough_code(col("PULocationID")))\
                    .withColumn("DO_borough_code", find_borough_code(col("DOLocationID")))\
                    .withColumn("week_of_year", find_week_of_year(col("PUtime")))\
                    .withColumn("day_of_week", find_day_of_week(col("PUtime")))\
                    .withColumn("part_of_day", find_part_of_day(col("PUtime")))\
                    #.drop("DOLocationID","PULocationID","PUtime")

processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2019 fhv
processing 2019 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv
processing 2020 fhv
processing 2020 fhvhv


In [32]:
#prints how much time has elapsed
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

2.471623900004488 seconds have elapsed
18.32405039999867 seconds have elapsed total


In [33]:
#█▀▀ █▀▀█ █  █ █▀▀▄ ▀▀█▀▀  ▀  █▀▀▄ █▀▀▀ 
#█   █  █ █  █ █  █   █   ▀█▀ █  █ █ ▀█ 
#▀▀▀ ▀▀▀▀  ▀▀▀ ▀  ▀   ▀   ▀▀▀ ▀  ▀ ▀▀▀▀ 
# As we have binned all the data into serveral differnt groups, we now need to count how many
# are in each group

#creates an empty array
dfc = {}
start_time2 = time.perf_counter()

#for each year
for year in years:
    dfc[year] = {}
    
    #for each month
    for month in range(covid_started[0], months_in_year + 1): 
        dfc[year][month] = {}
        
        #for each taxi service
        for taxi_i in range(0,len(taxi_file_names)):
            
            #clears array for data count
            dfc[year][month][taxi_i] = None
            
            #prints what stage count it up to
            print("counting " + str(year) + "-" + str(month)+" "+taxi_file_names[taxi_i])
            
            #if the data frame is not nothing
            if( dfm[year][month][taxi_i] ):
            
                #prints a dataframe of the count groupped by the binning columns
                dfc[year][month][taxi_i] = dfm[year][month][taxi_i]\
                    .groupBy("week_of_year","day_of_week","PU_borough_code","DO_borough_code","part_of_day")\
                    .count()\
                    .toPandas()
                
                #lost a lot of data mid way once so added this safe guard
                dfc[year][month][taxi_i].to_csv(data_dir + taxi_file_names[taxi_i] + processed_data_file + str(year) + "-" + str(month).zfill(2) + ".csv") 
            
            
            #prints how much time has elapsed
            print(str(time.perf_counter() - start_time2) + " seconds have elapsed")
            print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
            start_time2 = time.perf_counter()
        
    

counting 2019-2 fhv


78.87191539999912 seconds have elapsed
97.22273219999624 seconds have elapsed total
counting 2019-2 fhvhv


3605.051926699998 seconds have elapsed
3702.275276100001 seconds have elapsed total
counting 2019-3 fhv


22.79219720000401 seconds have elapsed
3725.0679205999986 seconds have elapsed total
counting 2019-3 fhvhv


3253.0290981 seconds have elapsed
6978.0972492 seconds have elapsed total
counting 2019-4 fhv


...

2188.367570599992 seconds have elapsed
65380.1884502 seconds have elapsed total
counting 2020-11 fhv


5.192939399988973 seconds have elapsed
65385.38190569999 seconds have elapsed total
counting 2020-11 fhvhv


1850.2864424 seconds have elapsed
67235.6688655 seconds have elapsed total
counting 2020-12 fhv


5.15673849999439 seconds have elapsed
67240.8258641 seconds have elapsed total
counting 2020-12 fhvhv


1860.051009600007 seconds have elapsed
69100.8770845 seconds have elapsed total


In [34]:
#prints how much time has elapsed
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

69082.5596543 seconds have elapsed
69100.8839692 seconds have elapsed total


In [35]:
#▀▀█▀▀ █▀▀█ █   █   █  █  ▀  █▀▀▄ █▀▀▀ 
#  █   █▄▄█ █   █   █▄▄█ ▀█▀ █  █ █ ▀█ 
#  ▀   ▀  ▀ ▀▀▀ ▀▀▀ ▄▄▄█ ▀▀▀ ▀  ▀ ▀▀▀▀ 
# Once all the bins are counted, we need to tally the corresponding bins into a useable dataframe

#function for counting a specific combonation of binning
def countthis_col(year, month, taxi_i, week, dayow, borough, part_of_day, borough_col):
    #print("\t subcount "+" ".join( [str(int) for int in [year, taxi_i]]))
    
    #returns the following sum from the counted dataframe
    return dfc[year][month][taxi_i]\
        .loc[
            (dfc[year][month][taxi_i]["week_of_year"] == week) &   #checks the week of the year matches
            (dfc[year][month][taxi_i]["day_of_week"] == dayow) &   #checks the day of the week matches
            (dfc[year][month][taxi_i][borough_col] == borough) &   #checks the borough matches
            (dfc[year][month][taxi_i]["part_of_day"] == part_of_day)  #checks the part of the day matches
        ]['count'].values.sum()   #calulates the sum of the counts

#this function is used to call the above function for both Pick up and drop off locations
def countthis(year, month, taxi_i, week, dayow, borough, part_of_day):
    
    #just returns the sum of the above function twice
    return countthis_col(year, month, taxi_i, week, dayow, borough, part_of_day, "PU_borough_code")\
         + countthis_col(year, month, taxi_i, week, dayow, borough, part_of_day, "DO_borough_code")
    
    
# returns an array of values or of 0s if it was empty  
def add_this_covid_data(day, stdar, borough, fhv, vals, length):
    if(vals.any()):
        output[10*(day - stdar) + 2 * borough + fhv].extend(vals[0])
    else:
        output[10*(day - stdar) + 2 * borough + fhv].extend([0] * length)
        
#output is just an array for now
output = [] 

#start day, moved here as is used for debugging periods
stdar = start_day #7*4+4


for day in range(stdar, days_in_2020):#100):#start_day+1):
    print(day)#start_day)
    #output.append([])
    datetimeO = datetime.datetime.strptime( " ".join( [str(int) for int in [day, covid_started[1]]]) ,"%j %Y" )
    dayow = int(datetimeO.__format__("%w"))
    week = int(datetimeO.__format__("%U"))
    
    for borough in range(0, len(covid_boroughs)):
        
        for fhv in range(0, 2):
            #[10*(day - start_day) + 2 * borough + fhv]
            output.append([
                dayow,
                week,
                borough,
                fhv
            ])
            
            
            
            
            for year in years:
                
                for part_of_day in range(0, len(day_time_breakdown)):
                    
                    count = 0
                                        
                    for month in range(covid_started[0], months_in_year + 1): 
                    #print("counting "+" ".join( [str(int) for int in [day, week, dayow, borough, fhv, year, part_of_day]]))
                        
                        #for fhv
                        if( fhv == 1):
                            count = 0
                            #for taxi_i in [fhv_i, hvf_i]:
                            #    if( dfm[year][month][taxi_i] ):
                            #        count += countthis(year, month, taxi_i, week, dayow, borough, part_of_day)
                            
                        #for non fhv
                        else:
                            for taxi_i in [ywl_i, grn_i]:
                                if( dfm[year][month][taxi_i] ):
                                    count += countthis(year, month, taxi_i, week, dayow, borough, part_of_day)
                                    
                    output[10*(day - stdar) + 2 * borough + fhv].append(count)#start_day) + 2 * borough + fhv].append(count)
            
            #adding relavent covid case numbers
            
            #adding general ones first
            add_this_covid_data(day, stdar, borough, fhv, 
                covid_cases.loc[covid_cases['date_of_interest'] == datetimeO][covid_of_interest].values,
                len(covid_of_interest)
            )
            
            #adding borough specific ones second
            add_this_covid_data(day, stdar, borough, fhv, 
                covid_cases.loc[covid_cases['date_of_interest'] == datetimeO][
                [covid_boroughs[borough] + "_" + suffix 
                 for suffix in covid_of_interest]]\
                .values,
                len(covid_of_interest)
            )
            
            
                    
            #adding relavent covid restrictions
            add_this_covid_data(day, stdar, borough, fhv, 
                covid_restrictions.loc[
                    covid_restrictions["date"] == int(datetime.datetime.timestamp(datetimeO)) #converts current rows date to unix and finds relavent restrictions
                ].drop("date", axis=1)
                .values,
                len(covid_restrictions.columns) - 1
            )


32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298


In [36]:
#prints how much time has elapsed
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

416.98177980000037 seconds have elapsed
69517.86608739999 seconds have elapsed total


In [37]:
#prints the current output array to see it looks alright
output

[[6,
  4,
  0,
  0,
  20042,
  23664,
  31693,
  42464,
  26358,
  35117,
  49654,
  60488,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  500.0,
  1.0,
  1.0,
  100000.0,
  1.0,
  0.0],
 [6,
  4,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  500.0,
  1.0,
  1.0,
  100000.0,
  1.0,
  0.0],
 [6,
  4,
  1,
  0,
  57787,
  55476,
  91866,
  132247,
  67044,
  67608,
  113468,
  151909,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  500.0,
  1.0,
  1.0,
  100000.0,
  1.0,
  0.0],
 [6,
  4,
  1,
  1,
  0,
  0,


In [38]:
# ██████╗  ██╗   ██╗ ████████╗ ██████╗  ██╗   ██╗ ████████╗ 
#██╔═══██╗ ██║   ██║ ╚══██╔══╝ ██╔══██╗ ██║   ██║ ╚══██╔══╝ 
#██║   ██║ ██║   ██║    ██║    ██████╔╝ ██║   ██║    ██║    
#██║   ██║ ██║   ██║    ██║    ██╔═══╝  ██║   ██║    ██║    
#╚██████╔╝ ╚██████╔╝    ██║    ██║      ╚██████╔╝    ██║    
# ╚═════╝   ╚═════╝     ╚═╝    ╚═╝       ╚═════╝     ╚═╝    
#


#the data has been outputted row by row in the following format
#
#data order, 
#date + general
#      dayow, week, borough,
#taxi 
#        fhv, for each years(
#                     for each partofday( 
#                            count))
#covid cases normal then borough
#     'case_count', 'probable_case_count','hospitalized_count', 'death_count', 'probable_death_count',
#       'case_count_7day_avg', 'all_case_count_7day_avg', 'hospitalized_count_7day_avg',
#       'death_count_7day_avg', 'all_death_count_7day_avg'
#restrictions
#       'pre_k_schools', 'elementary_schools', 'middle_schools',
#       'high_schools', 'construction', 'offices', 'phase_1', 'restaurants',
#       'outdoor_dining', 'phase_2', 'phase_3', 'phase_4',
#       'large_capacity_public', 'gatherings', 'broadway', 'movies',
#       'stadium_capcity', 'indoor_religious', 'curfew'#



#column names for final data frame
columns =  ["day", 
            "week", 
            "borough", 
            "fhv", 
            #"2018_night", 
            #"2018_morn",
            #"2018_arvo",
            #"2018_even",
            "2019_night", 
            "2019_morn",
            "2019_arvo",
            "2019_even",
            "2020_night", 
            "2020_morn",
            "2020_arvo",
            "2020_even",
            'total_case_count', 
            'total_probable_case_count',
            'total_hospitalized_count', 
            'total_death_count', 
            'total_probable_death_count',
            'total_case_count_7day_avg',
            'total_all_case_count_7day_avg', 
            'total_hospitalized_count_7day_avg',
            'total_death_count_7day_avg', 
            'total_all_death_count_7day_avg',
            'borough_case_count', 
            'borough_probable_case_count',
            'borough_hospitalized_count', 
            'borough_death_count', 
            'borough_probable_death_count',
            'borough_case_count_7day_avg',
            'borough_all_case_count_7day_avg', 
            'borough_hospitalized_count_7day_avg',
            'borough_death_count_7day_avg', 
            'borough_all_death_count_7day_avg',
            'pre_k_schools', 
            'elementary_schools', 
            'middle_schools',
            'high_schools', 
            'construction', 
            'offices', 
            'phase_1', 
            'restaurants',
            'outdoor_dining', 
            'phase_2', 
            'phase_3', 
            'phase_4',
            'large_capacity_public', 
            'gatherings', 
            'broadway', 
            'movies',
            'stadium_capcity', 
            'indoor_religious', 
            'curfew'
    ]

#converting output to a dataframe
dfo = pd.DataFrame(output, columns = columns)
dfo

,day,week,borough,fhv,2019_night,2019_morn,2019_arvo,2019_even,2020_night,2020_morn,...,phase_2,phase_3,phase_4,large_capacity_public,gatherings,broadway,movies,stadium_capcity,indoor_religious,curfew
0,6,4,0,0,20042,23664,31693,42464,26358,35117,...,1.0,1.0,1.0,1.0,500.0,1.0,1.0,100000.0,1.0,0.0
1,6,4,0,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,500.0,1.0,1.0,100000.0,1.0,0.0
2,6,4,1,0,57787,55476,91866,132247,67044,67608,...,1.0,1.0,1.0,1.0,500.0,1.0,1.0,100000.0,1.0,0.0
3,6,4,1,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,500.0,1.0,1.0,100000.0,1.0,0.0
4,6,4,2,0,83202,90911,163894,223892,84762,88811,...,1.0,1.0,1.0,1.0,500.0,1.0,1.0,100000.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,3,52,2,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,10.0,0.0,0.0,0.0,0.5,1.0
3336,3,52,3,0,0,0,0,0,10440,32004,...,1.0,1.0,1.0,1.0,10.0,0.0,0.0,0.0,0.5,1.0
3337,3,52,3,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,10.0,0.0,0.0,0.0,0.5,1.0
3338,3,52,4,0,0,0,0,0,826,3060,...,1.0,1.0,1.0,1.0,10.0,0.0,0.0,0.0,0.5,1.0


In [39]:
dfo.to_csv(data_dir + output_file)

In [40]:
output1 = output

In [41]:
print(str(time.perf_counter() - start_time) + " seconds have elapsed")
print(str(time.perf_counter() - start_time_og) + " seconds have elapsed total")
start_time = time.perf_counter()

0.7191846999921836 seconds have elapsed
69518.5856724 seconds have elapsed total
